In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
try:
    import emoji, soynlp, transformers
except:
    !pip install -U -q transformers emoji soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 20.6 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import re
from soynlp.normalizer import repeat_normalize

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
path = "/content/drive/MyDrive/2024경경대학술제/"
data_path = path + "data/"

# 모델 및 토크나이저 로드

In [ ]:
# # 데이터셋 로드
# df = pd.read_csv(data_path + 'data_mapleland_crawling_label_v1.csv')

# # 라벨을 숫자로 변환
# df.loc[df['라벨'].isin(['0', '1']), '라벨'] = 1
# df.loc[df['라벨'] == 'x', '라벨'] = 0

# # 데이터셋 분할
# train_set, test_set = train_test_split(df, test_size=0.1, stratify=df['라벨'])

# # CSV 파일로 저장
# train_set.to_csv(data_path + 'rmt_train.csv', index=False)
# test_set.to_csv(data_path + 'rmt_test.csv', index=False)

### 실험

In [ ]:
import numpy as np
# train_data에서 라벨이 0인 데이터를 찾기
label_0_indices = train_data[train_data['라벨'] == 0].index

# 무작위로 10000개의 인덱스 선택
random_indices_to_remove = np.random.choice(label_0_indices, size=10000, replace=False)

# 선택된 인덱스에 해당하는 데이터 제거
train_data_filtered = train_data.drop(random_indices_to_remove)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022", num_labels=2)

train_data = pd.read_csv(data_path + "rmt_train.csv")
val_data = pd.read_csv(data_path + "rmt_test.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data_filtered

,시각,이름,내용,위치,라벨
0,2024-04-05 13:16:50,님들아,황룡도 3상 삽니다 늑영3 자리팔아여,엘나스,0
1,2024-04-05 23:27:40,일병졍상길,스페판 정옵이상 삽니다,엘나스,0
2,2024-04-04 23:16:07,zl존도적xv,늑영3 자리 ㅅㅅ 말없구,엘나스,0
3,2024-04-06 00:02:37,Teddy,늑영3 자리삽니다~ 가격귓주세요,엘나스,0
4,2024-04-02 19:20:21,개똘개이,3상 골든힌켈 ㅍㅍ,엘나스,0
...,...,...,...,...,...
2861,2024-03-31 16:30:32,붉고기,아니 외노자 겹사 박네 머라 하니까 쨰고 ㅋㅋㅋ,엘나스,0
2862,2024-04-02 00:31:15,뚝딱애비,90이하 힌켈 색상관없이 ㅅㅅㅅ 선제시해주세요 망작 궁수 공장갑삽니다,엘나스,0
2863,2024-03-30 14:16:07,유논디,68 정법 죽1 2층 4시간 이상 지원합니다,엘나스,0
2864,2024-04-03 03:44:46,샐비즈,차벌1 솔플은 낭만임,엘나스,0


# 토크나이징 및 데이터셋 생성

In [ ]:
#전처리 함수 정의하여 변경 필요
def clean(x):
    #emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

train_data['내용'] = train_data['내용'].apply(clean)
val_data['내용'] = val_data['내용'].apply(clean)

train_labels = train_data['라벨'].values
train_texts = train_data['내용']

val_labels = val_data['라벨'].values
val_texts = val_data['내용']

print("가장 긴 문장의 길이:", max(len(sentence) for sentence in train_data['내용']))
print("가장 긴 문장의 길이:", max(len(sentence) for sentence in val_data['내용']))

가장 긴 문장의 길이: 50
가장 긴 문장의 길이: 50


In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64,
                            return_tensors='pt') # tensor 형태로 return

val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=64,
                            return_tensors='pt') # tensor 형태로 return

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}

    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

# 모델 설정 및 훈련

In [ ]:
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    macro_f1 = f1_score(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'macro_f1' : macro_f1,
        'precision': precision,
        'recall': recall
    }

# Optimizer 및 Learning Rate Scheduler 설정
optimizer = AdamW(model.parameters(), lr=5e-6)  # AdamW 옵티마이저를 사용하고 초기 학습률은 5e-5로 설정
scheduler = ExponentialLR(optimizer, gamma=0.5)

training_args = TrainingArguments(
    output_dir='/tmp/results',      #학습결과 저장경로
    evaluation_strategy = "epoch",  #에폭 단위로 검증
    num_train_epochs=3,             #학습 에폭
    per_device_train_batch_size=16, #train batch size
    per_device_eval_batch_size=64,  #test batch size
    logging_dir='/tmp/logs',        #학습 log 저장경로
    logging_steps=500,              #학습 log 기록 단위
    save_total_limit=2              #학습 결과 저장 최대갯수
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.704738,0.347552,0.091529,0.048604,0.783333
2,No log,0.704738,0.347552,0.091529,0.048604,0.783333
3,0.700100,0.704738,0.347552,0.091529,0.048604,0.783333


TrainOutput(global_step=540, training_loss=0.700204450112802, metrics={'train_runtime': 131.5225, 'train_samples_per_second': 65.373, 'train_steps_per_second': 4.106, 'total_flos': 216502370791560.0, 'train_loss': 0.700204450112802, 'epoch': 3.0})

In [ ]:
trainer.evaluate(eval_dataset=val_dataset)

{'eval_loss': 0.7309143543243408,
 'eval_accuracy': 0.1,
 'eval_f1': 0.08267997148966502,
 'eval_precision': 0.04318689501116903,
 'eval_recall': 0.9666666666666667,
 'eval_runtime': 8.0921,
 'eval_samples_per_second': 176.714,
 'eval_steps_per_second': 2.842,
 'epoch': 3.0}

In [ ]:
model_path = path + 'result/KcELECTRA_model'
tokenizer_path = path + 'result/KcELECTRA_tokenizer'
trainer.save_model(model_path)  # 모델 저장
tokenizer.save_pretrained(tokenizer_path)  # 토크나이저 저장

('/content/drive/MyDrive/2024경경대학술제/result/KcELECTRA_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/2024경경대학술제/result/KcELECTRA_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/2024경경대학술제/result/KcELECTRA_tokenizer/vocab.txt',
 '/content/drive/MyDrive/2024경경대학술제/result/KcELECTRA_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/2024경경대학술제/result/KcELECTRA_tokenizer/tokenizer.json')

# 검증

In [23]:
tokenizer_path = path + 'result/KcELECTRA_tokenizer'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [26]:
# 문장을 토큰화
text = "[ITEM][ITEM] 3상 // 인10 이속이카망 ㅍㅍ"
tokens = tokenizer.tokenize(text)
tokens

['[ITEM]',
 '[ITEM]',
 '3',
 '##상',
 '/',
 '/',
 '인',
 '##10',
 '이',
 '##속이',
 '##카',
 '##망',
 'ㅍ',
 '##ㅍ']

In [30]:
train_encodings = tokenizer(text, truncation=True, padding=True, max_length=64,
                            return_tensors='pt') # tensor 형태로 return
print(train_encodings)

{'input_ids': tensor([[    2, 54343, 54343,    23,  4161,    19,    19,  2703, 10548,  2700,
         17245,  4666,  4120,   370,  4619,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [25]:
# 추가할 스페셜 토큰 리스트를 정의합니다.
special_tokens = ["[ITEM]"]

# 토크나이저에 스페셜 토큰을 추가합니다.
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})

special_tokens_dict = tokenizer.special_tokens_map
special_tokens_list = tokenizer.all_special_tokens

print("Special tokens dictionary:", special_tokens_dict)
print("All special tokens:", special_tokens_list)

Special tokens dictionary: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[ITEM]']}
All special tokens: ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[ITEM]']


In [ ]:
test_data = val_data
predicted_list = []

# 저장된 모델과 토크나이저 불러오기
# loaded_model = ElectraForSequenceClassification.from_pretrained(model_path)
# loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

for text in test_data['내용']:
    model.eval()
    # 토큰화
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # 입력 데이터를 모델이 동작 중인 디바이스로 이동
    device = model.device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 문장별 logit을 합해서 클래스 분류(나중에 softmax 적용시 logit을 합할지 평균으로 할지에 따라 값이 바뀌니 주의)
    logits_sum = torch.sum(outputs.logits, dim=0)
    predicted_class = torch.argmax(logits_sum, dim=-1).cpu().numpy()

    # 예측된 클래스를 리스트에 추가
    predicted_list.append(predicted_class)

# 예측된 클래스를 DataFrame에 추가
test_data['predicted_class'] = predicted_list
test_data.head()

,시각,이름,내용,위치,라벨,predicted_class
0,2024-04-01 22:01:08,Unknown,104윙즈로 비슷한가격대 피완 구해봅니다 마력귓주세요,엘나스,0,1
1,2024-03-31 17:03:36,미니구름,남던1 자리 삽니다 귓주세요 ㅅㅅ,빅토리아,0,1
2,2024-04-04 01:37:14,나묵이형,남던2 자리있을까요 ?,빅토리아,0,1
3,2024-03-30 18:24:02,디그더,투방10퍼 16장 삽니다 장당75 277 헤네시스 택시앞,빅토리아,0,1
4,2024-04-05 23:56:57,서범호,인or럭 1상 다크 크리시스 800 삼,엘나스,0,1


In [ ]:
test_data[(test_data['라벨'] == 0) & (test_data['predicted_class'] == 0)]

,시각,이름,내용,위치,라벨,predicted_class
5,2024-04-02 07:59:46,노알,골든 아룬드 3상 // 인10 이속이카망 ㅍㅍ,엘나스,0,0
9,2024-04-03 01:32:22,나나아밍,태커형ㅊㅊㅊ 그래서 라이칸 일비없다 언제정리해줄거야,엘나스,0,0
10,2024-04-03 19:43:22,구십사년개띠,봉황위궁 공 90 3000에 급쳐함 흥정불가영,엘나스,0,0
12,2024-04-01 16:57:39,장제리,(68렙 법사 전신옷) 다크 레퀴에르 인6 럭12 떡작 499만에 팝니다 ㅍㅍ,엘나스,0,0
13,2024-04-05 05:11:01,왕포,죽숲1 왼1 하실 심마 프리님 구합니다 형님 오시죠,엘나스,0,0
...,...,...,...,...,...,...
1410,2024-04-05 15:47:48,명팔씨,"[팝니다] 스페판 공 100 힘5 , 다크클랜치 공6 가격 DM",엘나스,0,0
1412,2024-03-29 22:03:28,끼묘옹,공7노목 올메소로 삽니다,엘나스,0,0
1413,2024-04-01 05:19:04,bonnieclyde,늑영3 홀심알바구합니다 사례는 여기뜨는템 드릴게요 일비뺴고,엘나스,0,0
1416,2024-04-05 05:05:46,대하이햄동생,토비 110다삼 제섭 엘나스 운영자옆 교환주세요,엘나스,0,0


In [ ]:
test_data[test_data['라벨'] == 0]

,시각,이름,내용,위치,라벨,predicted_class
0,2024-04-01 22:01:08,Unknown,104윙즈로 비슷한가격대 피완 구해봅니다 마력귓주세요,엘나스,0,1
1,2024-03-31 17:03:36,미니구름,남던1 자리 삽니다 귓주세요 ㅅㅅ,빅토리아,0,1
2,2024-04-04 01:37:14,나묵이형,남던2 자리있을까요 ?,빅토리아,0,1
3,2024-03-30 18:24:02,디그더,투방10퍼 16장 삽니다 장당75 277 헤네시스 택시앞,빅토리아,0,1
4,2024-04-05 23:56:57,서범호,인or럭 1상 다크 크리시스 800 삼,엘나스,0,1
...,...,...,...,...,...,...
1425,2024-04-05 19:41:38,돌풍처럼,"죽숲1 76 프리스트 좌,우 2층 갑니다 아무곳이나 파티해요 귓주십셔! ! !!",엘나스,0,1
1426,2024-04-04 21:00:30,장흥이성규,장공 10퍼 5장 / 970 삽니다 / 귓 / 메랜지지 hump,엘나스,0,1
1427,2024-04-04 20:29:36,승솔이헤헤,82개 쪼렙련이 103사냥하는걸 방해하네?? 유추니<<<<<넌 시동얼씬도못한다,빅토리아,0,0
1428,2024-03-31 09:50:02,춘천왕감자,뇌전 팜팜 4000 귓귓,엘나스,0,1
